In [1]:
# Project 3 notebook

# Ideas from John:
# Does home country advantage exist in the Olympics?
# Which countries have the highest medal rates per competitor?
# Do some countries get disqualified more often than others?
# How have Olympic performances/scores changed over time?
# etc.

# https://olympics.com/en/